In [15]:
from qat.lang.AQASM import Program, H, X, CNOT, RX, I, RY, RZ, CSIGN #Gates
from qat.core import Observable, Term, Batch #Hamiltonian
import numpy as np
from qat.plugins import ScipyMinimizePlugin
import pickle

In [16]:
from qat.plugins import Junction
from qat.core import Result
from scipy.optimize import minimize
class Opto(Junction):
    def __init__(self, x0: np.ndarray = None, tol: float = 1e-8, maxiter: int = 25000, nbshots: int = 0,):
        super().__init__(collective=False)
        self.x0 = x0
        self.maxiter = maxiter
        self.nbshots = nbshots
        self.n_steps = 0
        self.energy_optimization_trace = []
        self.parameter_map = None
        self.energy = 0
        self.energy_result = Result()
        self.tol = tol
        self.c_steps = 0
        self.int_energy = []

    def run(self, job, meta_data):
        
        if self.x0 is None:
            self.x0 = 2*np.pi*np.random.rand(len(job.get_variables()))
            self.parameter_map = {name: x for (name, x) in zip(job.get_variables(), self.x0)}

        def compute_energy(x):
            job_bound =  job(** {v: xx for (v, xx) in zip(job.get_variables(), x)})
            self.energy = self.execute(job_bound)
            self.energy_optimization_trace.append(self.energy.value)
            self.n_steps += 1
            return self.energy.value

        def cback(intermediate_result):
            #fn =  compute_energy(intermediate_result)
            self.int_energy.append(intermediate_result.fun)
            self.c_steps += 1
            #return(fn)

        bnd = (0, 2*np.pi)
        bnds = tuple([bnd for i in range(len(job.get_variables()))])
        #res = minimize(compute_energy, x0 = self.x0, method='L-BFGS-B', bounds = bnds, callback = cback , options={'ftol': self.tol, 'disp': False, 'maxiter': self.maxiter})
        res = minimize(compute_energy, x0 = self.x0, method='COBYLA', bounds = bnds, options={'tol': self.tol, 'disp': False, 'maxiter': self.maxiter})
        en = res.fun
        self.parameter_map =  {v: xp for v, xp in zip(job.get_variables(), res.x)}
        self.energy_result.value = en
        self.energy_result.meta_data = {"optimization_trace": str(self.energy_optimization_trace), "n_steps": f"{self.n_steps}", "parameter_map": str(self.parameter_map), "c_steps" : f"{self.c_steps}", "int_energy": str(self.int_energy)}
        return (Result(value = self.energy_result.value, meta_data = self.energy_result.meta_data))

In [17]:
from qat.core.plugins import AbstractPlugin

class GaussianNoise(AbstractPlugin,):
    def __init__(self, p, ids, hamiltonian_matrix):
        self.p = p
        self.ids = ids
        self.hamiltonian_trace = np.trace(hamiltonian_matrix)/(np.shape(hamiltonian_matrix)[0])
        self.unsuccess = 0
        self.success = 0
        self.nb_pauli_strings = 0
        self.nbshots = 0

    @staticmethod
    def count_gates(in_circ):
        one_qb_gateset = ['H', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ']
        two_qb_gateset = ['CNOT', 'CSIGN']
        N_1qb = sum(in_circ.count(g) for g in one_qb_gateset)
        N_2qb = sum(in_circ.count(g) for g in two_qb_gateset)
        return N_1qb, N_2qb
    
    def compile(self, batch, _):
        self.nbshots =  batch.jobs[0].nbshots
        #nb_gates = batch.jobs[0].circuit.depth({'CNOT' : 2, 'RZ' : 1, 'H' : 1}, default = 1)
        n1, n2 = self.count_gates(batch.jobs[0].circuit)
        nb_gates = n1 + 2*n2 + self.ids
        self.success = abs((1-self.p)**nb_gates)
        self.unsuccess = (1-self.success)*self.hamiltonian_trace
        return batch 
    
    def post_process(self, batch_result):
        if batch_result.results[0].value is not None:
            for result in batch_result.results:
                if self.nbshots == 0:
                    noise =  self.unsuccess
                else: 
                    noise =  np.random.normal(self.unsuccess, self.unsuccess/np.sqrt(self.nbshots))
                result.value = self.success*result.value + noise
        return batch_result

In [18]:
nqbt = 5 # number of qubits
nruns = 0 #nbshots for observable sampling
nseeds = 10

In [19]:
dep = np.arange(1, 11, 1, dtype = int)

In [20]:
#Instantiation of Hamiltoniian
heisen = Observable(nqbt)
#Generation of Heisenberg Hamiltonian
for q_reg in range(nqbt-1):
    heisen += Observable(nqbt, pauli_terms = [Term(1., typ, [q_reg,q_reg + 1]) for typ in ['XX','YY','ZZ']])

In [21]:
#exact calculation for ground state
from qat.fermion import SpinHamiltonian

heisen_class = SpinHamiltonian(nqbits=heisen.nbqbits, terms=heisen.terms)
heisen_mat = heisen_class.get_matrix()

eigvals, eigvecs = np.linalg.eigh(heisen_mat)
g_energy = eigvals[0]
g_state = eigvecs[:,0]

In [22]:
from qat.qpus import get_default_qpu
qpu_ideal = get_default_qpu()

In [23]:
base = 1
expo = 5

In [24]:
#infidelity
F = base*(10**(-expo))

In [25]:
def total_ids_pl():
    return(42)

In [26]:
ress = {}

In [27]:
#variational gates
for ht in range(nseeds):
    np.random.seed(10*ht)
    res = []
    for ct in dep:
        #Variational circuit can only be constructed using the program framework
        qprog = Program()
        qbits = qprog.qalloc(nqbt)
        #variational parameters used for generating gates (permutation of [odd/even, xx/yy/zz])
        ao = [qprog.new_var(float, 'ao_%s'%i) for i in range(ct)]
        bo = [qprog.new_var(float, 'bo_%s'%i) for i in range(ct)]
        co = [qprog.new_var(float, 'co_%s'%i) for i in range(ct)]
        ae = [qprog.new_var(float, 'ae_%s'%i) for i in range(ct)]
        be = [qprog.new_var(float, 'be_%s'%i) for i in range(ct)]
        ce = [qprog.new_var(float, 'ce_%s'%i) for i in range(ct)]
        for q_index in range(nqbt):
            X(qbits[q_index])
        for q_index in range(nqbt):
            if not q_index%2 and q_index <= nqbt-1:
                H(qbits[q_index])
        for q_index in range(nqbt):
            if not q_index%2 and q_index <= nqbt-2:
                CNOT(qbits[q_index],qbits[q_index+1])
        for it in range(ct):
            for q_index in range(nqbt): #odd Rzz
                if q_index%2 and q_index <= nqbt-2:
                    CNOT(qbits[q_index],qbits[q_index+1])
                    RZ(ao[it-1]/2)(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
            for q_index in range(nqbt): #odd Ryy
                if q_index%2 and q_index <= nqbt-2:
                    RZ(np.pi/2)(qbits[q_index])
                    RZ(np.pi/2)(qbits[q_index+1])
                    H(qbits[q_index])
                    H(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
                    RZ(bo[it-1]/2)(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
                    H(qbits[q_index])
                    H(qbits[q_index+1])
                    RZ(-np.pi/2)(qbits[q_index])
                    RZ(-np.pi/2)(qbits[q_index+1])
            for q_index in range(nqbt): #odd Rxx
                if q_index%2 and q_index <= nqbt-2:
                    H(qbits[q_index])
                    H(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
                    RZ(co[it-1]/2)(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
                    H(qbits[q_index])
                    H(qbits[q_index+1])
            for q_index in range(nqbt): #even Rzz
                if not q_index%2 and q_index <= nqbt-2:
                    CNOT(qbits[q_index],qbits[q_index+1])
                    RZ(ae[it-1]/2)(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
            for q_index in range(nqbt): #even Ryy
                if not q_index%2 and q_index <= nqbt-2:
                    RZ(np.pi/2)(qbits[q_index])
                    RZ(np.pi/2)(qbits[q_index+1])
                    H(qbits[q_index])
                    H(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
                    RZ(be[it-1]/2)(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
                    H(qbits[q_index])
                    H(qbits[q_index+1])
                    RZ(-np.pi/2)(qbits[q_index])
                    RZ(-np.pi/2)(qbits[q_index+1])
            for q_index in range(nqbt): #even Rxx
                if not q_index%2 and q_index <= nqbt-2:
                    H(qbits[q_index])
                    H(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
                    RZ(ce[it-1]/2)(qbits[q_index+1])
                    CNOT(qbits[q_index],qbits[q_index+1])
                    H(qbits[q_index])
                    H(qbits[q_index+1])
        #circuit
        circuit = qprog.to_circ()
        ids_for_noise = 3 + ct*total_ids_pl()
        optimizer_scipy = ScipyMinimizePlugin(method="COBYLA",
                            tol=1e-6,
                            options={"maxiter": 25000},
#                                  x0=np.zeros(nqbt)
                                )
        stack_noisy = optimizer_scipy | GaussianNoise(F, ids_for_noise, heisen_mat) | qpu_ideal
        #job at hand, not the observable class but instantiated job
        job = circuit.to_job(observable = heisen, nbshots = 0)
        res_noisy = stack_noisy.submit(job)
        res.append(res_noisy)
    ress['%s'%ht] = res

In [28]:
#with open('global_noisy_seeds/noisy-%s_%s.pkl'%(base, expo), 'wb') as file:
with open('Results/global_noisy_gates_seeds_assumptions_COBY_scipy/noisy-%s_%s.pkl'%(base, expo), 'wb') as file:
    pickle.dump(ress, file)